<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Optimizing with Decision Frames  

Work in Progress 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.37"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.0 Microsoft.SemanticKernel, 1.37.0

Using Azure OpenAI Service


### Step 2 - Decision Frames (Thinking Hats) Scenario for a Major Purchase 

The code below will list the potential decision frames (using thinking hats frames) that may be appropriate to form different approaches for purchasing a house. Generative AI can be used to easily "put on a hat" (approach a problem with a specific decision frame) much easier than a human decision maker.  

In [3]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions: 
List an approach for each of the six hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Edward de Bono's Six Thinking Hats is a powerful decision-making framework that ensures you evaluate a situation from multiple perspectives. Here's how you can use each of the six hats in the context of purchasing a new home:

---

### **1. White Hat (Facts and Information)**
Focus: Data, facts, and available information. What do you know, and what do you need to know?

- **Approach:**  
  - Gather information about your budget, mortgage options, and financial standing.  
  - Research the housing market, property prices, interest rates, and market trends.  
  - Assess the needs of your household: location, size, school proximity, work commute, amenities, etc.  
  - List must-have features and nice-to-have features.  
  - Identify knowledge gaps: What questions do you need to ask realtors, banks, or home inspectors?  

---

### **2. Red Hat (Feelings and Intuition)**  
Focus: Emotions, hunches, and gut reactions.

- **Approach:**  
  - Consider how each potential home makes you feel. Do

This can be enhanced further by having GenAI suggest which of the decision frames might be the most useful for a high-stakes decision. Therefore, the prompt above can be optimized further by listing all the decision frames and then priorotizing which ones will have the most impact and influence on the house purchase decision.  

This allows the Artificial Intelligence automation to focus on the optimization of the decision process and the human on the actual decision selection and execution!

In [4]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:
List an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list suggest which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Certainly! Using Edward de Bono's **Six Thinking Hats** framework, we can systematically approach the decision-making process for purchasing a new home. Let’s start by listing the hats in the **prioritized order based on their impact and influence** on this specific decision. I'll then suggest which hats to focus on for optimal results:

---

### **1. White Hat (Information and Data Focus)**
- **Purpose**: Focuses on gathering all relevant facts, data, and information about the house, neighborhood, finances, and market trends.
- **Approach**:
  - Understand your budget, mortgage options, and long-term financial implications.
  - Review neighborhood amenities, schools, proximity to work, and crime rates.
  - Analyze housing market trends: Is it a buyer's or a seller's market? Home appreciation/value trends.
  - Check structural conditions of the home: inspections, maintenance costs, and potential repairs.
  
- **Impact**: This is critical because purchasing a home is a long-term financi

So far, (given the house purchase decision) Generative AI has helped create decision frames with the six thinking hats framework, rank them from most impact & influence from most to least and select the top 3 decision frames. The next logical step is to leverage Generative AI to apply the selected 3 decision frames for the house purchase decision.  

This can be done in two different ways:  
1) **Single Decision Prompt:** Instruct (prompt) the GenAI model "think internally" on the decision frames, ranking, selecting the top decision frames and then finally output the applied decision frames for the house purchase decision.
2) **Workflow Decision Prompts:** This approach uses multiple prompts together to arrive at a decision approach. This breaks the problem down into smaller components and executes each component in sequence. The value is that each step can be optimized individually and each intermediate output is clearly available for analysis. This is ideal for sufficiently important decision problems.  

#### Single Decision Prompt 

In [ ]:
// Set the overall system prompt to behave like a Decision Intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Decision Frames Prompt that lists the six thinking hats approach to purchasing a house 
var decisionFramesListPrompt = """
You are planning on purchasing a new home. 
Use Edward de Bono's Six thinking hats on how to approach the decision-making process. 

Instructions:

<Do Not Show these steps>
INTERNALLY THINK about an approach for each of the six hats in the order of impact & influence from most to least. 
From the priorotized list INTERNALLY PROCESS which thinking hats (decision frames) to focus on. 
It can be one of the hats or any combination of the thinking hats. 
Let's call that internal list "selected decision frames"
</Do Not Show these steps>

Only output these steps to the user: 
Finally, from the internal "selected decision frames", create an approach for each for purchasing a house.  
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(decisionFramesListPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Here’s an approach based on the "selected decision frames" for purchasing a house:

### Selected Decision Frames and Their Approach:

#### 1. **White Hat (Facts and Information)**:
   - **Gathering Data**: Start by collecting all relevant information about the housing market, including current prices, types of homes within your budget, neighborhoods, schools, commute times, and amenities nearby.
   - **Financial Analysis**: Review your finances—calculate your budget, explore mortgage options, and understand additional costs like property taxes, closing costs, and maintenance expenses.
   - **Understanding Trends**: Research market trends to assess whether it's a buyer's or seller's market. Analyze home value appreciation rates for potential investment growth.

#### 2. **Yellow Hat (Optimism and Opportunities)**: 
   - **Envisioning Positives**: Imagine how this decision could serve your long-term goals. For example, owning a home could mean building equity, increasing financial stabili